En este notebook se aplicara la fase 2 del workflow jerarquico.
Ya no clasificaremos la macro categoria [cs, math, physics, ...]
Ahora nos centraremos en clasificar las probabilidades de las subcategorias....

En este caso, en la fase 2, no solo tendremos un modelo b (TENDREMOS UNO POR CADA MACRO CATEGORIA), ya que cuando se entrenan las subcategorias no son toda juntas en el mismo saco. 
las subcategorias de cs se entrenan entre ellas, no mas.
las subcategorias de math se entrenan entre ellas, no mas.
y asi para todas.

Esta fase hara lo siguiente.
Recorrer cada area de estudio (fisica, matematicas, ciencias computacionales, ...) por separado
cada que llega a un area, discrimina lo demas.
entrenara un mlp para distinguir subcategorias
se guardara de nuevo un .pkl

In [1]:
import os
import numpy as np
import pandas as pd
import joblib
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

#parametros
TEST_SIZE = 0.2
RANDOM_STATE = 42

#paths
CLEANED = "/Users/morenx/Downloads/mt/codigos/cleanedv3.csv"
EMBEDD = "/Users/morenx/Downloads/mt/embeddingspecter.npy" 
LABEL = "/Users/morenx/Downloads/mt/labelspecter.csv"
#data 

cleaned = pd.read_csv(CLEANED)
X = np.load(EMBEDD)
df = pd.read_csv(LABEL)

In [8]:
macro_categorias = df['macro_cat'].unique()
print(f"Se entrenarán nuestros modelos especialistas para: {macro_categorias}\n")

for macro in macro_categorias:
    print(f"procesando: {macro.upper()}")
    #filtramos el area
    indices = df[df["macro_cat"] == macro].index

    #subconjunto 
    df_sub = df.loc[indices].copy()
    X_sub = X[indices] # embeddings correspondientes del area

    conteo = df_sub['primary_cat'].value_counts()
    clases_validas = conteo[conteo >= 10].index

    mask_validos = df_sub['primary_cat'].isin(clases_validas)
    #nos saltamos una clase si es que tiene pocos papers
    if len(df_sub) < 50:
        print(f"nos saltamos: {macro} tiene muy pocos datos ({len(df_sub)}) para entrenar.")
        continue

    print(f"   > Total Papers: {len(df_sub)}")
    print(f"   > Subcategorías a aprender: {len(clases_validas)}")
    
    ### preparamos el entrenamiento
    le_especialista = LabelEncoder()
    y_encoded = le_especialista.fit_transform(df_sub['primary_cat'])

    #data split local por clase
    X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(
            X_sub, y_encoded, 
            test_size=TEST_SIZE, 
            random_state=RANDOM_STATE, 
            stratify=y_encoded
        )
    # construccion del mlp
    mlp_b = MLPClassifier(
        hidden_layer_sizes=(256, 128),
        activation='relu',
        solver='adam',
        max_iter=100,
        early_stopping=True,
        random_state=42
    )
    
    mlp_b.fit(X_train_b, y_train_b)
    
    # guardamos cada modelo
    NOMBRE = macro.replace(' ', '_').replace('&', 'and')
    NOMBRE_ARCHIVO = f"modelo_B_{NOMBRE}.pkl"
    
    paquete = {
        'modelo': mlp_b,
        'encoder': le_especialista,
        'macro_cat': macro,
        'test_accuracy': mlp_b.score(X_test_b, y_test_b)
    }
    
    joblib.dump(paquete, NOMBRE_ARCHIVO)
    print(f" guardado exitosamente: {NOMBRE_ARCHIVO}\n")

print("TODOS LOS MODELOS ESTÁN LISTOS")
    

Se entrenarán nuestros modelos especialistas para: ['Computer Science' 'Economics & Finance' 'Electrical Engineering'
 'Mathematics' 'Physics' 'Quantitative Biology' 'Statistics']

procesando: COMPUTER SCIENCE
   > Total Papers: 20000
   > Subcategorías a aprender: 40
 guardado exitosamente: modelo_B_Computer_Science.pkl

procesando: ECONOMICS & FINANCE
   > Total Papers: 9504
   > Subcategorías a aprender: 12
 guardado exitosamente: modelo_B_Economics_and_Finance.pkl

procesando: ELECTRICAL ENGINEERING
   > Total Papers: 14153
   > Subcategorías a aprender: 4
 guardado exitosamente: modelo_B_Electrical_Engineering.pkl

procesando: MATHEMATICS
   > Total Papers: 20000
   > Subcategorías a aprender: 35
 guardado exitosamente: modelo_B_Mathematics.pkl

procesando: PHYSICS
   > Total Papers: 20000
   > Subcategorías a aprender: 53
 guardado exitosamente: modelo_B_Physics.pkl

procesando: QUANTITATIVE BIOLOGY
   > Total Papers: 19886
   > Subcategorías a aprender: 10
 guardado exitosamente

YA OBTENIDOS TODOS LOS MODELOS, CREAREMOS NUESTRO SISTEMA FINAL...

HAREMOS LA PRIMERA PRUEBA DE NUESTRO CODIGO...

In [3]:
import joblib
import numpy as np
import os

class ArxivSystem:
    def __init__(self, models_path="./", umbral_cs=0.30):
        self.path = models_path
        self.umbral_cs = umbral_cs

        # modelo A (fase 1)
        try:
            ruta_modelo_a = '/Users/morenx/Downloads/mt/codigos/model_a_specter.pkl' 

            if not os.path.exists(ruta_modelo_a):
                 ruta_modelo_a = os.path.join(self.path, '/Users/morenx/Downloads/mt/codigos/model_a_specter.pkl')

            sistema_a = joblib.load(ruta_modelo_a)
            self.model_a = sistema_a['modelo']
            self.encoder_a = sistema_a['encoder']
            print("Fase 1 cargada en memoria.")
        except FileNotFoundError:
            raise Exception(f" No se cargó el modelo Fase 1.")

        #  modelos especialistas en cada area
        self.specialists_cache = {} 

        # umbral de optimizacion de CS
        self.idx_cs = self.encoder_a.transform(['Computer Science'])[0]

    def get_specialist(self, macro_cat):
        safe_name = macro_cat.replace(' ', '_').replace('&', 'and')
        return f"modelo_B_{safe_name}.pkl"

    def load_specialist(self, macro_cat):
        """
        Carga un especialista solo si no está ya en memoria.
        """
        if macro_cat in self.specialists_cache:
            return self.specialists_cache[macro_cat]
        
        # Es nuevo, cargarlo del disco
        filename = self.get_specialist(macro_cat)
        filepath = os.path.join(self.path, filename)
        
        if not os.path.exists(filepath):
            return None # No existe especialista para esta clase 
            
        print(f" cargando especialista: {macro_cat}...")
        try:
            sistema_b = joblib.load(filepath)
            # Guardamos cache
            self.specialists_cache[macro_cat] = sistema_b
            return sistema_b
        except Exception as e:
            print(f"Error al leer {filename}: {e}")
            return None

    def predict(self, vector_embedding, return_full_path=False):
        """
        Inferencia jerárquica para nuestros embeddings
        """
        #  forma (1, 768)
        vector = vector_embedding.reshape(1, -1)
        
        # FASE 1_ MODELO A [MACRO CATEGORIAS]
        probs = self.model_a.predict_proba(vector)[0]
        
        # Umbral para probabilidades
        pred_idx = np.argmax(probs)
        
        # Prioridad CS
        if probs[self.idx_cs] > self.umbral_cs:
            pred_idx = self.idx_cs
            
        macro_cat = self.encoder_a.inverse_transform([pred_idx])[0]
        
        # FASE 2_ MODELO B [ESPECIALISTAS]
        specialist = self.load_specialist(macro_cat)
        
        if specialist is None:
            # PEOR ESCENARIO: El modelo A predijo algo para lo que no entrenamos sub-modelo
            final_pred = f"{macro_cat} (General)"
        else:
            model_b = specialist['modelo']
            encoder_b = specialist['encoder']
            
            pred_sub_idx = model_b.predict(vector)[0]
            final_pred = encoder_b.inverse_transform([pred_sub_idx])[0]
            
        if return_full_path:
            return {'macro': macro_cat, 'sub': final_pred}
        
        return final_pred

    def batch_predict(self, X_matrix):
        """Opción para predecir muchos papers a la vez"""
        return [self.predict(vec) for vec in X_matrix]

In [4]:
PATH_MODELOS = "/Users/morenx/Downloads/mt/modelos"
arxiv_bot = ArxivSystem(models_path=PATH_MODELOS)

print("Probando Inferencia ")
#  paper de prueba aleatorio
paper_test = X[500] 

# revisamos etiqueta real
real_label = df.iloc[500]['primary_cat']

# pred
resultado = arxiv_bot.predict(paper_test, return_full_path=True)

print(f"Realidad: {real_label}")
print(f"Predicción: {resultado}")

#  Validación 10 papers
print("\nPrueba Rápida (10 papers) ")
indices_random = np.random.choice(len(X), 10)
X_sample = X[indices_random]
y_real_sample = df.iloc[indices_random]['primary_cat'].values

y_preds = arxiv_bot.batch_predict(X_sample)

for real, pred in zip(y_real_sample, y_preds):
    acierto = "acertado" if real == pred else "erroneo"
    print(f"{acierto} Real: {real} | Pred: {pred}")

Fase 1 cargada en memoria.
Probando Inferencia 
 cargando especialista: Computer Science...
Realidad: cs.DM
Predicción: {'macro': 'Computer Science', 'sub': 'cs.FL'}

Prueba Rápida (10 papers) 
 cargando especialista: Electrical Engineering...
 cargando especialista: Quantitative Biology...
 cargando especialista: Statistics...
 cargando especialista: Mathematics...
acertado Real: eess.SY | Pred: eess.SY
erroneo Real: eess.AS | Pred: cs.CL
acertado Real: eess.AS | Pred: eess.AS
acertado Real: q-bio.MN | Pred: q-bio.MN
acertado Real: q-bio.NC | Pred: q-bio.NC
acertado Real: stat.AP | Pred: stat.AP
acertado Real: math.RT | Pred: math.RT
acertado Real: cs.SI | Pred: cs.SI
erroneo Real: stat.ML | Pred: cs.LG
acertado Real: q-bio.NC | Pred: q-bio.NC


In [ ]:
import numpy as np
import joblib
import pandas as pd

print("EVALUACIÓN FINAL: TOP-3 ACCURACY ")

#1k paper de prueba
n_test = 1000
indices_random = np.random.choice(len(X), n_test, replace=False)
X_sample = X[indices_random]

raw_labels = df.iloc[indices_random]['primary_cat'].values
legacy_mapping = {
    'alg-geom': 'math.AG', 'funct-an': 'math.FA', 'q-alg': 'math.QA', 'dg-ga': 'math.DG',
    'geo-alg': 'math.AG', 'cmp-lg': 'cs.CL', 'adap-org': 'nlin.AO', 'comp-gas': 'nlin.CG',
    'chao-dyn': 'nlin.CD', 'patt-sol': 'nlin.PS', 'solv-int': 'nlin.SI',
    'supr-con': 'cond-mat.supr-con', 'acc-phys': 'physics.acc-ph', 'chem-ph': 'physics.chem-ph',
    'mat-ph': 'math-ph'
}
y_real = pd.Series(raw_labels).replace(legacy_mapping).values

#  inicializamos el sistema manual (para acceder a probabilidades) 
aciertos_top3 = 0
aciertos_top1 = 0 

print(f"Evaluando {n_test} papers...")

for i, vector in enumerate(X_sample):
    # fase 1
    res_fase1 = arxiv_bot.predict(vector, return_full_path=True) 
    macro_cat = res_fase1['macro']
    
    # fase 2
    specialist = arxiv_bot.load_specialist(macro_cat)
    
    if specialist is None:
        continue # no hubo un modelo b especialista asignado (ojala no pase )
        
    model_b = specialist['modelo']
    encoder_b = specialist['encoder']
    
    # Obtenemos probabilidades de todas las subcategorías
    probs = model_b.predict_proba(vector.reshape(1, -1))[0]
    
    # Obtenemos los índices de las 3 más altas
    top3_indices = np.argsort(probs)[-3:][::-1]
    top3_classes = encoder_b.inverse_transform(top3_indices)
    
    # Verificamos
    real_label = y_real[i]
    
    if real_label == top3_classes[0]: # Fue la #1
        aciertos_top1 += 1
        aciertos_top3 += 1
    elif real_label in top3_classes: # Estaba en la #2 o #3
        aciertos_top3 += 1

print(f"EXACT MATCH (Top-1): {aciertos_top1/n_test:.4f}")
print(f"TOP-3 ACCURACY:    {aciertos_top3/n_test:.4f}")


EVALUACIÓN FINAL: TOP-3 ACCURACY 
Evaluando 1000 papers...
 cargando especialista: Physics...
 cargando especialista: Economics & Finance...
EXACT MATCH (Top-1): 0.6330
TOP-3 ACCURACY:    0.8100


Teniendo en cuenta que nuestros datos cuentan con mas de 150 clases (muchas de ellas superpuestas) es un resultado bastante decente, aunque con algo mas de tiempo, se podria mejorar.

INTERPETACION DE RESULTADOS...

Considero que la razon por la cual el modelo no tiene una precision mas alta, no se debe a que el modelo no haya aprendido como se debe, el problema no es que el modelo no distinga un paper de fisica con uno de economia, el problema es que se debe de confundir (por ejemplo, cs.LG[machine learning] y cs.AI[artificial inteligence]).
al darle 3 oportunidades, acerto casi el 80% de las veces. Con esto podria concluir que el modelo entiende semantica perfectamente, solo se confunde por los prefijos de arxiv

In [7]:
import plotly.graph_objects as go


valores_top1 = [0.6330] # 63.3%
valores_top3 = [0.8100] # Tu 81%
azar = [0.007] 

categorias = ['Sistema Jerárquico']

fig = go.Figure()

# Barra del Azar (Referencia)
fig.add_trace(go.Bar(
    x=categorias, y=azar, name='Azar (Random)', marker_color='gray'
))

# Barra Top-1
fig.add_trace(go.Bar(
    x=categorias, y=valores_top1, name='Exact Match (Top-1)', marker_color='indianred',
    text=[f"{v*100:.1f}%" for v in valores_top1], textposition='auto'
))

# Barra Top-3
fig.add_trace(go.Bar(
    x=categorias, y=valores_top3, name='Top-3 Accuracy', marker_color='teal',
    text=[f"{v*100:.1f}%" for v in valores_top3], textposition='auto'
))

fig.update_layout(
    title='Top-1 vs Top-3',
    yaxis_title='Accuracy',
    barmode='group'
)

fig.show()

Esta grafica sera la prueba de que mi proyecto fue un exito. 

La barra gris, aunque no se vea mucho, son los papers que no se supieron clasificar y se asignaron al azar

La barra roja, ese 60.9% representa las veces que el modelo acerto a la primera con seguridad. 
(por ejemplo, el paper de prueba era cs.AI y el modelo dijo con seguridad que es cs.AI)
este modelo sufrio bastante por la ambiguedad de arxiv, ya que si el modelo dijo que el paper era cs.LG (machine learning) y el paper era stat.ML (machine learning pero en estadisica) el fallo contara como un cero

La barra verde, ese 78.2% es el caso en el que mejor le va al modelo. representa las veces que la respuesta correcta estaba entre las 3 opciones que el modelo considero mas probables, es prueba de que el sistema SI es inteligente.

Esta diferencia entre barra roja y verde significa una cosa, no es que el modelo se equivoque profundamente (por ejemplo, diciendo que un paper es de finanzas cuando en realidad es de biologia), es por que duda de cosas muy parecidas, como ya se conto anteriormente.
